# Backtesting ML Classification-Based

## Load the model

In [1]:
import pickle

In [2]:
with open('models/model_dt_microsoft_linkedin_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [3]:
model_dt

DecisionTreeClassifier(max_depth=15)

In [14]:
model_dt.predict(X=?)

SyntaxError: invalid syntax (4146865962.py, line 1)

## Load the data

In [4]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Simulated.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,daily_diff,daily_diff_direction,buy,buy_sell
Date,,,,,,,,,
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.882095,UP,1,1
2016-12-12,56.803028,57.244073,56.711145,57.124622,20198100,0.183765,UP,1,0
...,...,...,...,...,...,...,...,...,...
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,4.649994,UP,1,0
2023-03-16,265.209991,276.559998,263.279999,276.200012,54768800,10.760010,UP,1,0


## Backtesting.py Library

### Create your Strategy Class

In [5]:
from backtesting import Backtest, Strategy

/Users/sota/miniforge3/envs/backtesting/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [6]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [7]:
df_explanatory.iloc[-1:, :]

,Open,High,Low,Close,Volume
Date,,,,,
2023-03-16,265.209991,276.559998,263.279999,276.200012,54768800


In [8]:
df.daily_diff_direction[-1:]

Date
2023-03-16    UP
Name: daily_diff_direction, dtype: object

In [9]:
class SimpleClassificationUD(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[-1:, :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [10]:
bt = Backtest(df_explanatory, SimpleClassificationUD,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [11]:
results = bt.run()

### Interpret backtesting results

In [12]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-09 00:00:00
End,2023-03-16 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],99.873096
Equity Final [$],2112.721079
Equity Peak [$],10056.430816
Return [%],-78.872789


### Visualize the strategy performance

In [14]:
bt.plot(filename='reports_backtesting/backtesting_classification.html')

Row(id='2426', ...)